In [5]:
import h5py
import pandas

In [4]:
k = "//cluster/home/simondi/simondi/tix/data/screening_data/cells_sample10.h5"

In [6]:
data = pandas.HDFStore(k)

In [7]:
data

<class 'pandas.io.pytables.HDFStore'>
File path: //cluster/home/simondi/simondi/tix/data/screening_data/cells_sample10.h5
/group_cossart-listeria-d-p-g-1-dz01-1k_cells                    frame        (shape->[3840,175])
/group_cossart-listeria-d-p-g-1-dz02-1k_cells                    frame        (shape->[3840,175])
/group_cossart-listeria-d-p-g-1-dz03-1k_cells                    frame        (shape->[3840,175])
/group_cossart-listeria-d-p-g-1-dz04-1k_cells                    frame        (shape->[3840,175])
/group_cossart-listeria-d-p-g-1-dz05-1k_cells                    frame        (shape->[3840,175])
/group_cossart-listeria-d-p-g-1-dz06-1k_cells                    frame        (shape->[3831,175])
/group_cossart-listeria-d-p-g-1-dz07-1k_cells                    frame        (shape->[3840,175])
/group_cossart-listeria-d-p-g-1-dz08-1k_cells                    frame        (shape->[3840,175])
/group_cossart-listeria-d-p-g-1-dz09-1k_cells                    frame        (shape->[3832,17